# Quant Research Notebook: Daily Feature Extraction for OHLCV Stock Data
This notebook extracts daily model-ready features from cleaned OHLCV data.
Important: the cleaned dataset in `dataset/cleaned/` is *not* smoothed/filtered. Any filtering methods are used only as *features* and are exported separately to `dataset/features/`.


## Feature Catalog (Exact Columns Exported)

This notebook exports a large set of *parameterized* features (multiple windows/periods).
To satisfy the research requirement of documenting *every extracted feature*, the following table programmatically lists the exact exported column names and attaches the matching math definition and parameter values.


In [10]:
# Build a feature catalog from the exporter logic
import re
import pandas as pd

FEATURE_DOCS = []

def add_doc(name, family, math, notes=''):
    FEATURE_DOCS.append({'feature': name, 'family': family, 'math': math, 'notes': notes})

# Returns
add_doc('ret_1d', 'returns', 'R_t = C_t/C_{t-1} - 1')
add_doc('logret_1d', 'returns', 'r_t = ln(C_t/C_{t-1})')
add_doc('excess_ret_1d', 'returns', 'R_t^{excess} = R_t - r_{f,t}')

# Lags
add_doc('logret_lag_1', 'lag', 'logret_{t-1}')
add_doc('logret_lag_5', 'lag', 'logret_{t-5}')
add_doc('ret_lag_1', 'lag', 'ret_{t-1}')
add_doc('ret_lag_5', 'lag', 'ret_{t-5}')

# Multi-horizon returns
add_doc('ret_5d', 'returns', 'C_t/C_{t-5} - 1')
add_doc('ret_21d', 'returns', 'C_t/C_{t-21} - 1')
add_doc('logret_5d', 'returns', 'ln(C_t) - ln(C_{t-5})')
add_doc('logret_21d', 'returns', 'ln(C_t) - ln(C_{t-21})')

# Cumulative returns
add_doc('cumret_5d', 'cumulative return', 'prod_{i=0..4}(1+R_{t-i}) - 1')
add_doc('cumret_21d', 'cumulative return', 'prod_{i=0..20}(1+R_{t-i}) - 1')

# Differencing on log price
add_doc('diff_log_close_1', 'differencing', 'Delta ln(C_t) = ln(C_t) - ln(C_{t-1})')
add_doc('diff_log_close_2', 'differencing', 'Delta^2 ln(C_t) = Delta(Delta ln(C_t))')

# Rolling statistics on log returns
for w in (5, 10, 20, 60):
    add_doc(f'logret_roll_mean_{w}', 'rolling mean', f'mu_t^(w) = (1/w) * sum_{{i=0..w-1}} r_{{t-i}}')
    add_doc(f'logret_roll_var_{w}', 'rolling variance', f'sigma_t^2(w) = (1/w) * sum (r - mu)^2')
    add_doc(f'logret_roll_std_{w}', 'rolling std', f'sigma_t(w) = sqrt(sigma_t^2(w))')
    add_doc(f'logret_roll_min_{w}', 'rolling min', f'min(r_{{t-w+1..t}})')
    add_doc(f'logret_roll_max_{w}', 'rolling max', f'max(r_{{t-w+1..t}})')

# Volume rolling stats + z-scores
add_doc('log_volume', 'transform', 'ln(V_t)')
for w in (5, 20, 60):
    add_doc(f'volume_roll_mean_{w}', 'rolling mean', f'mu_V,t^(w) = (1/w) * sum V_{{t-i}}')
    add_doc(f'volume_roll_std_{w}', 'rolling std', f'sigma_V,t^(w)')
    add_doc(f'volume_zscore_{w}', 'z-score', f'z_t = (V_t - mu_V,t^(w)) / sigma_V,t^(w)')

# Scaling / normalization
add_doc('logret_zscore_20', 'z-score', 'z_t = (r_t - mu_t^(20)) / sigma_t^(20)')
add_doc('close_minmax_20', 'min-max scaling', '(C_t - min_t^(20)) / (max_t^(20) - min_t^(20))')
add_doc('volume_minmax_20', 'min-max scaling', '(V_t - min_V,t^(20)) / (max_V,t^(20) - min_V,t^(20))')

# Volatility
add_doc('atr_14', 'ATR', 'TR_t = max(H-L, |H-C_{t-1}|, |L-C_{t-1}|); ATR = SMA_14(TR)')
add_doc('realized_vol_20', 'realized volatility', 'sqrt(252) * Std_{20}(logret_1d)')

# Technical indicators
add_doc('rsi_14', 'RSI', 'RSI = 100 - 100/(1+RS), RS=AvgGain/AvgLoss (Wilder)')
add_doc('sma_ratio_20', 'SMA (paper)', 'SMAhat = C_t / SMA_20(C)_t (arXiv:2412.15448 Eq. 5)')
add_doc('ema_ratio_20', 'EMA (paper)', 'EMAhat = C_t / EMA_20(C)_t (arXiv:2412.15448 Eq. 7)')
for c in ('macd', 'macd_signal', 'macd_hist'):
    add_doc(f'macd_{c}', 'MACD', 'MACD=EMA12-EMA26; Signal=EMA9(MACD); Hist=MACD-Signal')
add_doc('rmacd_12_26_9', 'MACD (paper)', 'rMACD=(MACD-SIG)/(0.5*(|MACD|+|SIG|)) (arXiv:2412.15448 Eq. 9)')
for c in ('bb_mid', 'bb_upper', 'bb_lower', 'bb_bandwidth', 'bb_percent_b'):
    add_doc(f'bb_{c}', 'Bollinger', 'Mid=SMA20; Upper/Lower=Mid +/- 2*Std20; %B=(C-L)/(U-L)')
add_doc('roc_10', 'ROC', 'ROC_t = C_t/C_{t-10} - 1')
for c in ('stoch_k', 'stoch_d'):
    add_doc(f'stoch_{c}', 'stochastic', '%K = 100*(C-LL14)/(HH14-LL14); %D=SMA3(%K)')
add_doc('cci_20', 'CCI (paper)', 'CCI=(p-SMA(p))/(0.015*MAD) with p=(H+L+C)/3 (arXiv:2412.15448 Eq. 19)')

# Volume indicators
add_doc('obv', 'OBV', 'OBV_t = OBV_{t-1} +/- V_t based on sign(C_t-C_{t-1})')
add_doc('obv_roc_10', 'OBV ROC', 'ROC_t = OBV_t/OBV_{t-10} - 1')
add_doc('wrobv_20', 'WROBV (paper)', 'WROBV=sum_{i=0..19}OBV_{t-i}/sum_{i=0..19}V_{t-i} (arXiv:2412.15448 Eq. 17)')
add_doc('ad_line', 'Accumulation/Distribution', 'CLV=((C-L)-(H-C))/(H-L); AD_t=AD_{t-1}+CLV*V')

# Trend indicators
for c in ('plus_di', 'minus_di', 'adx', 'adx_raw'):
    add_doc(f'adx_{c}', 'ADX', '+DI=100*Sm(+DM)/ATR; -DI=100*Sm(-DM)/ATR; ADX=Sm(DX)')
for c in ('aroon_up', 'aroon_down'):
    add_doc(f'aroon_{c}', 'Aroon', 'AroonUp=100*(N-days_since_HH)/N; AroonDown=100*(N-days_since_LL)/N')

# Ichimoku
for c in ('ichimoku_conv', 'ichimoku_base', 'ichimoku_span_a', 'ichimoku_span_b', 'ichimoku_lagging'):
    add_doc(f'ichimoku_{c}', 'Ichimoku', 'Conv=(HH9+LL9)/2; Base=(HH26+LL26)/2; spans shifted +/-26')

# Fibonacci levels
for c in ('fib_swing_high', 'fib_swing_low', 'fib_23_6', 'fib_38_2', 'fib_50_0', 'fib_61_8', 'fib_76_4'):
    add_doc(f'fib_{c}', 'Fibonacci', 'Levels derived from rolling swing high/low: H - p*(H-L)')
add_doc('fib_retr_60', 'Fibonacci (paper)', 'R(t)=(H_60(t)-C_t)/(H_60(t)-L_60(t)) (arXiv:2412.15448 Eq. 14)')

# Market structure event indicators (swing_window=3)
add_doc('bos', 'market structure', 'BOS: Close breaks prior swing high/low (bullish=+1, bearish=-1)', 'swing_window=3')
add_doc('choch', 'market structure', 'CHoCH: BOS opposite to prevailing structure direction (bullish=+1, bearish=-1)', 'swing_window=3')
add_doc('mss', 'market structure', 'MSS: CHoCH confirmation via next BOS in new direction (bullish=+1, bearish=-1)', 'swing_window=3')

# Filter-based features (cleaning-pipeline filters used as features)
FILTERS = [
    'sma_5', 'ema_12', 'wma_5', 'savgol_11_2', 'kalman_r0p1_q0p01', 'wiener_5',
    'spectral_0p1', 'lms_mu1e-4_taps5', 'lattice_demo'
]
for f in FILTERS:
    add_doc(f'filt_close_{f}', 'filter feature', 'Filtered close: C~_t')
    add_doc(f'filt_resid_{f}', 'filter feature', 'Residual: e_t = C_t - C~_t')
    add_doc(f'filt_logret_{f}', 'filter feature', 'Filtered log return: ln(C~_t/C~_{t-1})')

# Raw context columns
add_doc('close', 'raw context', 'C_t')
add_doc('volume', 'raw context', 'V_t')

catalog = pd.DataFrame(FEATURE_DOCS).sort_values(['family', 'feature']).reset_index(drop=True)
catalog


,feature,family,math,notes
0,adx_adx,ADX,+DI=100*Sm(+DM)/ATR; -DI=100*Sm(-DM)/ATR; ADX=...,
1,adx_adx_raw,ADX,+DI=100*Sm(+DM)/ATR; -DI=100*Sm(-DM)/ATR; ADX=...,
2,adx_minus_di,ADX,+DI=100*Sm(+DM)/ATR; -DI=100*Sm(-DM)/ATR; ADX=...,
3,adx_plus_di,ADX,+DI=100*Sm(+DM)/ATR; -DI=100*Sm(-DM)/ATR; ADX=...,
4,atr_14,ATR,"TR_t = max(H-L, |H-C_{t-1}|, |L-C_{t-1}|); ATR...",
...,...,...,...,...
116,log_volume,transform,ln(V_t),
117,logret_zscore_20,z-score,z_t = (r_t - mu_t^(20)) / sigma_t^(20),
118,volume_zscore_20,z-score,"z_t = (V_t - mu_V,t^(w)) / sigma_V,t^(w)",
119,volume_zscore_5,z-score,"z_t = (V_t - mu_V,t^(w)) / sigma_V,t^(w)",


In [11]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath('..'))
# If you re-run this notebook after editing feature code, reload modules
import importlib
try:
    import src.cleaning.ts_models as ts_models
    importlib.reload(ts_models)
except ModuleNotFoundError as e:
    if getattr(e, 'name', '') == 'statsmodels':
        ts_models = None
        print('Optional dependency missing: statsmodels (ts_models disabled).')
    else:
        raise
import src.export_features as export_features
importlib.reload(export_features)


<module 'src.export_features' from '/home/anivarth/college/quant-task/src/export_features.py'>

## Input Data Assumptions (Daily OHLCV)
We assume each asset has daily rows with columns: `Open`, `High`, `Low`, `Close`, `Volume`, indexed by `Date`.
The cleaned dataset in `dataset/cleaned/` is produced via imputation-only steps (no smoothing filters).


In [12]:
sample = pd.read_csv('../dataset/cleaned/Asset_001.csv', parse_dates=['Date']).set_index('Date')
sample.head()


,Open,High,Low,Close,Volume
Date,,,,,
2016-01-25,29.178415,29.181290,28.514486,28.580592,249449990
2016-01-26,28.721415,28.994458,28.186822,28.738659,361581962
2016-01-27,27.603374,27.772948,26.827351,26.850345,642328247
2016-01-28,26.956690,27.166502,26.554308,27.042913,268157355
2016-01-29,27.244108,27.977016,27.117645,27.977016,310239413


## Feature Families
Each feature below includes: (1) intuition, (2) math definition, (3) implementation notes for daily data.
All features are computed per asset per day, and exported to `dataset/features/`.


## Convention: Notation and Data

We use daily OHLCV notation:
- $O_t, H_t, L_t, C_t$ = open, high, low, close on day $t$
- $V_t$ = traded volume on day $t$
- Returns are derived from $C_t$ unless stated otherwise.

Most rolling features use a trailing window of length $w$ and are undefined (NaN) until $w$ observations exist.


## 1) Return Features (Simple, Log, Excess)

### Intuition
Raw prices are not comparable across assets. Returns normalize by the price level: a $5$ move on a $10$ stock is large, but on a $1000$ stock is small.

### Math
- Simple return: $R_t = \frac{C_t}{C_{t-1}} - 1$
- Log return: $r_t = \ln\left(\frac{C_t}{C_{t-1}}\right) = \ln(C_t) - \ln(C_{t-1})$
- Excess return: $R_t^{excess} = R_t - r_{f,t}$ where $r_{f,t}$ is the per-day risk-free rate.

### Notes
- Log returns are additive over time, which helps with modeling and aggregation.
- Excess returns isolate compensation over the risk-free baseline.


## 2) Lag Features (Momentum)

### Intuition
Momentum effects suggest recent returns can contain predictive information. Lagged returns give the model direct access to recent history.

### Math
For a series $x_t$, the $k$-lag feature is: $x_{t-k}$.


## 3) Rolling Statistics (Mean/Var/Min/Max/Std)

### Intuition
Rolling statistics summarize local behavior: trend (mean), dispersion (variance/std), and extremes (min/max).

### Math
For a trailing window of length $w$:
- Rolling mean: $\mu_t^{(w)} = \frac{1}{w}\sum_{i=0}^{w-1} x_{t-i}$
- Rolling variance: $\sigma_t^{2,(w)} = \frac{1}{w}\sum_{i=0}^{w-1}(x_{t-i}-\mu_t^{(w)})^2$
- Rolling min/max: $\min_{i=0..w-1} x_{t-i}$, $\max_{i=0..w-1} x_{t-i}$

### Notes
Initial periods have NaNs until enough history accumulates.


## 4) Differencing (Trend Removal on Semilog Scale)

### Intuition
Differencing removes slow-moving trends and makes a series more stationary. On a semilog plot, differencing log-price corresponds to log returns.

### Math
- First difference: $\Delta x_t = x_t - x_{t-1}$
- Second difference: $\Delta^2 x_t = (x_t-x_{t-1}) - (x_{t-1}-x_{t-2})$
Applied to log price: $\Delta \ln(C_t) = \ln(C_t/C_{t-1})$


## 5) Rolling Z-Scores for Volume

### Intuition
A volume z-score tells how unusual today's activity is compared to the recent past (liquidity and attention proxy).

### Math
$z_t = \frac{V_t - \mu_t^{(w)}}{\sigma_t^{(w)}}$ where $\mu_t^{(w)}$ and $\sigma_t^{(w)}$ are rolling mean/std of volume over window $w$.


## 6) Volatility Indicators (Rolling Vol, ATR)

### Intuition
Volatility measures dispersion of price moves and is central to risk-based strategies. ATR uses OHLC to quantify daily range volatility.

### Math
- Realized volatility (annualized): $\sigma_{t}^{(w)} = \sqrt{252}\cdot \text{Std}(r_{t-w+1:t})$
- True Range: $TR_t = \max(H_t-L_t, |H_t-C_{t-1}|, |L_t-C_{t-1}|)$
- ATR: $ATR_t^{(w)} = \frac{1}{w}\sum_{i=0}^{w-1} TR_{t-i}$


## 7) Technical + Momentum Indicators

This section covers common technical indicators used as features: MACD, RSI, Bollinger Bands, ROC, and Stochastic Oscillator.


### MACD
- MACD line: $MACD_t = EMA_{12}(C)_t - EMA_{26}(C)_t$
- Signal line: $Signal_t = EMA_{9}(MACD)_t$
- Histogram: $Hist_t = MACD_t - Signal_t$

EMA recursion: $EMA_t = \alpha C_t + (1-\alpha)EMA_{t-1}$ with $\alpha=2/(n+1)$.


### RSI (14)
RSI measures the balance of recent gains vs losses.

- $RS = \frac{AvgGain}{AvgLoss}$
- $RSI = 100 - \frac{100}{1+RS}$

Wilder smoothing is implemented via exponential smoothing with $\alpha=1/14$.


### Bollinger Bands (20, 2)
- Mid: $Mid_t = SMA_{20}(C)_t$
- Upper/Lower: $Mid_t \pm 2\cdot Std_{20}(C)_t$
Also exported: Bandwidth and %B position within bands.


### ROC (Rate of Change)
$ROC_t^{(n)} = \frac{C_t}{C_{t-n}} - 1$


### Stochastic Oscillator (14,3)
$\%K_t = 100\cdot \frac{C_t - L^{(14)}_t}{H^{(14)}_t - L^{(14)}_t}$, where $H^{(14)}_t$ is rolling highest-high and $L^{(14)}_t$ is rolling lowest-low.
$\%D_t = SMA_3(\%K)_t$.


## 8) Volume-Based Indicators (OBV, A/D Line)

### OBV Intuition
Volume can precede price. OBV accumulates volume conditioned on price direction.

### OBV Math
Let $OBV_0 = 0$. For $t>0$:
- if $C_t > C_{t-1}$: $OBV_t = OBV_{t-1} + V_t$
- if $C_t < C_{t-1}$: $OBV_t = OBV_{t-1} - V_t$
- else: $OBV_t = OBV_{t-1}$

We also export OBV rate-of-change because absolute OBV magnitude is less meaningful.

### Divergence (OBV)
- Price rising while OBV falls/plateaus: potential distribution (selling pressure).
- Price falling/sideways while OBV rises: potential accumulation (buying pressure).

### Limitations (OBV)
- A single day of extreme volume can dominate the series.
- OBV uses direction only; magnitude of price moves is ignored.
- Best used alongside other indicators (e.g., RSI/MFI).

### Accumulation/Distribution (A/D) Line
Close Location Value: $CLV_t = \frac{(C_t-L_t)-(H_t-C_t)}{H_t-L_t}$
Money Flow Volume: $MFV_t = CLV_t\cdot V_t$
A/D line: $AD_t = \sum_{i\le t} MFV_i$

### Trend Confirmation (A/D)
- If price rises and A/D rises, buying pressure supports the uptrend.
- If price falls but A/D rises, buyers may be stepping in (potential reversal).


## 9) Trend Strength (ADX, Aroon)

### ADX
ADX measures trend strength (non-directional). We export +DI, -DI, and ADX.

Directional movement: $+DM_t = H_t - H_{t-1}$, $-DM_t = L_{t-1} - L_t$ with standard gating rules.
True range and ATR normalize these to +DI/-DI, then $DX_t = 100\cdot \frac{|+DI_t - -DI_t|}{|+DI_t + -DI_t|}$, and $ADX$ is a smoothed DX.

Interpretation (common heuristic):
- $ADX < 20$: weak/absent trend
- $ADX > 25$: trend strength increasing

### Aroon (25)
AroonUp: $100\cdot \frac{N - \text{periods since highest high}}{N}$
AroonDown: $100\cdot \frac{N - \text{periods since lowest low}}{N}$


## 12) Filter-Based Features (From the Cleaning Pipeline)

### Why include filters as *features*?
Filtering can create alternative views of the same price process (trend + residual decomposition). In quant modeling, we often feed both the filtered signal and the residual (original minus filtered) to let the model learn which regime benefits from filtering.

Important: For stock data, aggressive filtering can blur true jumps/gaps and may degrade predictive features. This notebook keeps filtering strictly in the feature layer; the cleaned dataset is not filtered.

### Features exported
For each filter we export three series:
- Filtered close: $\tilde{C}_t$
- Residual: $e_t = C_t - \tilde{C}_t$
- Filtered log return: $\ln(\tilde{C}_t/\tilde{C}_{t-1})$

### Filters included (and math)
1) SMA (5): $\tilde{C}_t = \frac{1}{k}\sum_{i=0}^{k-1} C_{t-i}$ (trailing window).
2) EMA (12): $\tilde{C}_t = \alpha C_t + (1-\alpha)\tilde{C}_{t-1}$, $\alpha=2/(k+1)$.
3) WMA (5): $\tilde{C}_t = \frac{\sum_{i=1}^k w_i C_{t-k+i}}{\sum_{i=1}^k w_i}$ with linear weights $w_i=i$.
4) Savitzky-Golay (11,2): fit a degree-2 polynomial over a trailing window and evaluate at the most recent point.
5) 1D Kalman filter: state model $x_t = x_{t-1} + w_t$, observation $z_t = x_t + v_t$; update via Kalman gain $K_t$.
6) Wiener filter: minimum MSE linear filter; causal implementation.
7) Spectral low-pass: Causal Butterworth recursive filter ($4^{th}$ order).
8) LMS adaptive filter: $w_{t+1}=w_t + 2\mu e_t x_t$ (causal recursive update).
9) Lattice filter (demo): recursive forward/backward prediction error update using reflection coefficients $k_m$.

AR/ARMA model-based filters are excluded because they often emit convergence/non-stationarity warnings on stock series.


## 10) Ichimoku Cloud

We export conversion, base, leading spans A/B (shifted forward), and lagging span (shifted back).
- Conversion (9): $(HH_9 + LL_9)/2$
- Base (26): $(HH_{26} + LL_{26})/2$
- Span A: (Conversion + Base)/2 shifted +26
- Span B: $(HH_{52} + LL_{52})/2$ shifted +26
- Lagging: Close shifted +26 (Chikou value representing past price at current time)


## 11) Fibonacci Retracement Levels (Rolling Swing)

Using a rolling swing high/low over a window, we compute retracement levels: 38.2%, 50%, 61.8%.
If $H^{(w)}_t$ is rolling high and $L^{(w)}_t$ rolling low, range $R_t = H^{(w)}_t - L^{(w)}_t$.
Then level 61.8%: $H^{(w)}_t - 0.618\cdot R_t$ (and similarly for 50%, 38.2%).


## 12) Market Structure Events (BOS / CHoCH / MSS)

We export three discrete event indicators (values in {-1, 0, 1}):
- `bos`: break of structure (bullish +1 when Close breaks above prior swing high; bearish -1 when breaks below prior swing low)
- `choch`: change of character (BOS opposite to the prevailing structure direction)
- `mss`: market structure shift confirmation (after CHoCH, the next BOS in the new direction)

Swing highs/lows are detected using a trailing local-extrema window (`swing_window=3`) and confirmed with a lag of `swing_window` periods.


## Export Features (All Assets)
The exporter writes per-asset daily feature CSVs and a combined file:
- `dataset/features/Asset_XXX.csv`
- `dataset/features/all_features.parquet` (if `pyarrow` or `fastparquet` is installed)
- `dataset/features/all_features.csv` (fallback if no Parquet engine is available)

Risk-free rate is parameterized; default is 0.0 unless you set it explicitly.


In [13]:
from src.export_features import export_feature_data
export_feature_data(cleaned_dir='../dataset/cleaned', output_dir='../dataset/features', risk_free_rate_annual=0.0)


Parquet engine missing (pyarrow/fastparquet). Wrote combined CSV instead: ../dataset/features/all_features.csv


In [14]:
feat_sample = pd.read_csv('../dataset/features/Asset_001.csv', parse_dates=['Date']).set_index('Date')
feat_sample.head()


,ret_1d,logret_1d,excess_ret_1d,logret_lag_1,logret_lag_5,ret_lag_1,ret_lag_5,ret_5d,ret_21d,logret_5d,...,filt_resid_spectral_0p1,filt_logret_spectral_0p1,filt_close_lms_mu1e-4_taps5,filt_resid_lms_mu1e-4_taps5,filt_logret_lms_mu1e-4_taps5,filt_close_lattice_demo,filt_resid_lattice_demo,filt_logret_lattice_demo,close,volume
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28.568686,NaN,147.167302,-118.586710,NaN,28.580592,0.000000,NaN,28.580592,249449990.0
2016-01-26,0.005531,0.005515,0.005531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28.641189,2.102446,147.167302,-118.428643,0.0,31.768202,-3.029543,0.105738,28.738659,361581962.0
2016-01-27,-0.065706,-0.067965,-0.065706,0.005515,NaN,0.005531,NaN,NaN,NaN,NaN,...,26.455787,1.398224,147.167302,-120.316957,0.0,31.385692,-4.535347,-0.012114,26.850345,642328247.0
2016-01-28,0.007172,0.007146,0.007172,-0.067965,NaN,-0.065706,NaN,NaN,NaN,NaN,...,25.963696,1.006224,147.167302,-120.124389,0.0,31.957946,-4.915033,0.018069,27.042913,268157355.0
2016-01-29,0.034542,0.033958,0.034542,0.007146,NaN,0.007172,NaN,NaN,NaN,NaN,...,25.683599,0.753807,147.167302,-119.190286,0.0,32.817241,-4.840225,0.026533,27.977016,310239413.0
